In [2]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Input
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.saving import load_model

#### Data

In [3]:
path_to_file = '/home/alvaro/tf_templates/DATA/Quijote.txt'

In [4]:
text = open(path_to_file, 'r', encoding='utf-8').read()

#### Text vectorization

In [5]:
vocab = sorted(set(text))
print(vocab)
print(f'Vocabulary size: {len(vocab)}')

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü', '—']
Vocabulary size: 92


In [7]:
chars_to_ids_layer = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
ids_to_chars_layer = tf.keras.layers.StringLookup(vocabulary=chars_to_ids_layer.get_vocabulary(), invert=True, mask_token=None)

In [9]:
def text_from_ids(ids):
    joined_tensor = tf.strings.reduce_join(ids_to_chars_layer(ids), axis=-1)
    return joined_tensor.numpy().decode('utf-8')

In [10]:
all_ids = chars_to_ids_layer(tf.strings.unicode_split(text, 'UTF-8'))
print(text[:20])
print(all_ids.numpy()[:20])

El ingenioso hidalgo
[26 58  2 56 60 54 52 60 56 61 65 61  2 55 56 51 48 58 54 61]


#### Create Batch of Sequences

In [12]:
seq_len = 500
total_num_seq = len(text) // (seq_len+1)
total_num_seq

4211

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [14]:
sequences_dataset = ids_dataset.batch(seq_len+1, drop_remainder=True)

In [15]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [16]:
dataset = sequences_dataset.map(split_input_target)

In [18]:
# Shows an example
for input_example, target_example in dataset.take(1):
    print("Input sequence:")
    print(f"  IDs: {input_example.numpy()}")
    print(f"  Text: {text_from_ids(input_example)}\n")
    print("Target sequence:")
    print(f"  IDs: {target_example.numpy()}")
    print(f"  Text: {text_from_ids(target_example)}\n")

Input sequence:
  IDs: [26 58  2 56 60 54 52 60 56 61 65 61  2 55 56 51 48 58 54 61  2 51 61 60
  2 37 67 56 57 61 66 52  2 51 52  2 58 48  2 33 48 60 50 55 48  1 62 61
 64  2 33 56 54 67 52 58  2 51 52  2 24 52 64 68 48 60 66 52 65  2 39 48
 48 68 52 51 64 48  1 26 58  2 56 60 54 52 60 56 61 65 61  2 55 56 51 48
 58 54 61  2 51 61 60  2 37 67 56 57 61 66 52  2 51 52  2 58 48  2 33 48
 60 50 55 48  1 40 48 65 48  1 40 52 65 66 56 59 61 60 56 61  2 51 52  2
 58 48 65  2 52 64 64 48 66 48 65  1 26 58  2 38 52 70  1 22 58  2 25 67
 63 67 52  2 51 52  2 23 84 57 48 64  1 36 64 88 58 61 54 61  1 22 58  2
 58 56 49 64 61  2 51 52  2 51 61 60  2 37 67 56 57 61 66 52  2 51 52  2
 58 48  2 33 48 60 50 55 48  1 37 67 52  2 66 64 48 66 48  2 51 52  2 58
 48  2 50 61 60 51 56 50 56 88 60  2 70  2 52 57 52 64 50 56 50 56 61  2
 51 52 58  2 53 48 59 61 65 61  2 55 56 51 48 58 54 61  2 51 61 60  2 37
 67 56 57 61 66 52  2 51 52  2 58 48  2 33 48 60 50 55 48  1 37 67 52  2
 66 64 48 66 48  2 51 52  2 

2025-05-12 22:05:11.413002: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [19]:
batch_size = 128
buffer_size = 10000

In [20]:
dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
print(f'Dataset prepared for training: {dataset}')

Dataset prepared for training: <_PrefetchDataset element_spec=(TensorSpec(shape=(128, 500), dtype=tf.int64, name=None), TensorSpec(shape=(128, 500), dtype=tf.int64, name=None))>


#### Creating the Model

In [21]:
vocab_size = chars_to_ids_layer.vocabulary_size()
embed_dim = 256
rnn_neurons = 1024

In [33]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size, seq_len):
    model = Sequential([
        Input(shape=(seq_len,), batch_size=batch_size),
        Embedding(input_dim=vocab_size, output_dim=embed_dim),
        GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])

    return model

In [34]:
model = create_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    rnn_neurons=rnn_neurons,
    batch_size=batch_size,
    seq_len=seq_len
)

In [35]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (128, 500, 256)        │        23,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_6 (GRU)                     │ (128, 500, 1024)       │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (128, 500, 93)         │        95,325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,057,437 (15.48 MB)

 Trainable params: 4,057,437 (15.48 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Test 1 batch
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print('(batch_size, sequence_length, vocab_size)')
    print(example_batch_predictions.shape)

(batch_size, sequence_length, vocab_size)
(128, 500, 93)


In [37]:
model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=True))

#### Training

In [38]:
epochs = 150

In [39]:
early_stop = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

In [40]:
history = model.fit(dataset, epochs=epochs, callbacks=[early_stop])

Epoch 1/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - loss: 3.4759
Epoch 2/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 2.2008
Epoch 3/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 2.0075
Epoch 4/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 1.8599
Epoch 5/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 1.7308
Epoch 6/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 1.6120
Epoch 7/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 76ms/step - loss: 1.5109
Epoch 8/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 1.4406
Epoch 9/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 7s 189ms/step - loss: 1.3771
Epoch 10/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 1.3314
Epoch 11/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - loss: 1.2928
Epoch 12/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - loss: 1.2585
Epoch 13/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 1.2331
Epoch 14/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - loss: 1.2091
Epoch 15/150
32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - l

In [41]:
model.save_weights('quijote.weights.h5')

### Generate new text

In [42]:
model_gen = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1, seq_len=1)

In [43]:
model_gen.load_weights('quijote.weights.h5')

In [44]:
model_gen.build()

In [45]:
model_gen.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (1, 1, 256)            │        23,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_7 (GRU)                     │ (1, 1, 1024)           │     3,938,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (1, 1, 93)             │        95,325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,057,437 (15.48 MB)

 Trainable params: 4,057,437 (15.48 MB)

 Non-trainable params: 0 (0.00 B)

In [46]:
def generate_text(model, start_seed, gen_size=500, temperature=1.0):
    num_generate = gen_size

    input_eval_chars = tf.strings.unicode_split(start_seed, 'UTF-8')
    input_eval_ids = chars_to_ids_layer(input_eval_chars)
    input_eval = tf.expand_dims(input_eval_ids, 0)

    text_generated = []

    for i in range(num_generate):
        predictions = model_gen(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(ids_to_chars_layer(predicted_id).numpy().decode('utf-8'))

    return (start_seed + ''.join(text_generated))

In [47]:
print(generate_text(model_gen, start_seed="En un lugar de La Mancha", gen_size=1000, temperature=1.0))

En un lugar de La Mancha, venga lo que viniere, y repara usar de otro ánimo para quitársele la cabeza; y, en quejo, le ponen a tomar tus buenos escuderos con gentil donaire y honesta y contada o cabrir, y si escuchada y le pida la celada pluma, hecho roto, y si no puede ser mandallos y en obras la esposa Miguel de Micomicon y si quisiéredes para ellos. Pero, con todo esto, sube tan encamido y en suerte ser, pues la fortuna cosa que no pueda pasar por ahora, porque no creyeses que este buen hombre le corrió en otra acerca. El ventero tenía tan al vivo, que estaba decíandonse arrogante y sin haberte dido:
cargan con la mistro castillo al dinero, y por el lugar un hijo de la Amor, a quien lo dure el cielo contra quien no te la diga.

Un negocio se supo su huésped y le dijo:

— Para que huye de tener es que sé yo quiero, yo responde cuenta con este caballero, que por donde penséis que sabes mi compañía deben de ser cosa de guiar, como si los hay, daré la vuelta de la pastora Marcela, con o

In [48]:
print(generate_text(model_gen, start_seed="En un lugar de La Mancha", gen_size=1000, temperature=0.5))  # More predictable

En un lugar de La Mancha se volvieron a proseguir su camino de aquel loco que fue don Quijote; de la cual dio Sancho que departiesen y el barbero había conocido cuantos escribiese. Y, como la historia convenía y comenzó a toda la cuenta de su deseo, se contentó don Quijote con su escudo, se despidió de sus huéspedes y de todos los caballeros pasados:

— Si vuestra merced, señor caballero andante una cosa tan contado —dijo a este punto Sancho Panza—, mujer de ingenioso hidalgo don Quijote de la Mancha cuando Dios se atreve los traslos. Dese más que don Antonio tenía en alguna grande ahecha suerte de sus madrechorimos correos que en los vientos y principales padecen cuatro de sus melandras; la cual, contento con los pies a la cabeza, había de estar por vencido de la duquesa hasta llegar a un primo herido al agradeciéndolo que le esperase aquí, porque veía que llevaban a ser herida no permitiese; y, después que le dejaban ven, o porque no era de mejor para estar suspenso, contó lo que el 